In [1]:
import pandas
import numpy

In [7]:
data = pandas.read_excel("train_GB_KachestvoPrepodovaniya.xlsx", engine="calamine", sheet_name="Результат запроса")

In [57]:
data = data[["ID урока", "Дата старта урока", "Роль пользователя", "Текст сообщения", "Дата сообщения"]]
data.dropna()

,ID урока,Дата старта урока,Роль пользователя,Текст сообщения,Дата сообщения
0,307751.0,2024-03-06 10:02:38,user,"winningConditions = [\n [0, 1, 2],\n [3,...",2024-03-06 10:53:03
1,307752.0,2024-03-13 09:51:49,user,https://diresnode.com/; 2024-03-13T10:12:22.638Z,2024-03-13 10:12:22
2,321813.0,2024-03-02 12:56:26,user,пока; 2024-03-02T14:29:01.773Z,2024-03-02 14:29:01
3,321813.0,2024-03-02 12:56:26,user,пока; 2024-03-02T14:29:07.513Z,2024-03-02 14:29:07
4,321813.0,2024-03-02 12:56:26,user,досвидания; 2024-03-02T14:29:08.331Z,2024-03-02 14:29:08
...,...,...,...,...,...
121360,437109.0,2024-03-31 14:00:12,user,"Не поверите, он все это время на зарядке(",2024-03-31 15:21:12
121361,437109.0,2024-03-31 14:00:12,user,"Видимо, отложить. Спасибо!; 2024-03-31T15:22:2...",2024-03-31 15:22:22
121362,437224.0,2024-03-30 06:54:04,user,Здравствуйте!; 2024-03-30T07:27:52.900Z,2024-03-30 07:27:52
121364,437258.0,2024-03-30 06:55:50,user,Здравствуйте; 2024-03-30T07:02:37.652Z,2024-03-30 07:02:37


In [ ]:
# !pip install transformers sentencepiece --quiet
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-tiny-sentiment-balanced'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()
    
def get_sentiment(text, return_type='label'):
    """ Calculate sentiment of a text. `return_type` can be 'label', 'score' or 'proba' """
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()[0]
    if return_type == 'label':
        return model.config.id2label[proba.argmax()]
    elif return_type == 'score':
        return proba.dot([-1, 0, 1])
    return proba

In [46]:
text = 'И никто ничо не понял!	'
# classify the text
print(get_sentiment(text, 'label'))

negative


In [81]:
useless = ["пока", "до свидания", "досвидания", "салам", "привет", "здравствуйте", "#", "/", "\\", "1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]

def filter(text):
    clean = True
    for trash in useless:
        if trash in text.lower():
            clean = False
            
    if clean:
        if ";" in text:
            normal = text.split(";")
            return normal[0]
        else:
            return text
    
texts = data["Текст сообщения"].dropna()
no_dublicates = list(set(texts.map(filter).dropna().to_list()))

len(no_dublicates)

81530

In [85]:
numpied = numpy.array(no_dublicates)

In [90]:
from sentence_transformers import SentenceTransformer
sentences = ["Звука нет", "Не работает звук"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)

[[ 1.26032397e-01 -1.04202375e-01  2.89876938e-01 -1.75656527e-01
  -1.52347758e-01 -1.57574803e-01  1.29800111e-01 -9.75484252e-02
  -4.05911095e-02  9.70070343e-03  1.03682831e-01 -1.59332037e-01
   3.00769303e-02 -1.97448462e-01  2.19731137e-01  1.83498308e-01
   2.21625537e-01  3.05084381e-02 -4.21761349e-02  5.90147562e-02
   2.05524802e-01  1.49826929e-01  9.01318491e-02  2.35095695e-02
   1.01703145e-01  1.45775646e-01 -1.28701657e-01  5.45360986e-03
  -1.78062394e-01  4.52945521e-03 -1.87051226e-03 -5.88739812e-02
   2.18581602e-01 -4.00287397e-02  5.36365211e-02  9.87779051e-02
   1.28254205e-01 -2.73599587e-02 -4.94584411e-01 -1.55334827e-02
   2.81061325e-02 -4.11134213e-02 -8.19696933e-02 -1.72583058e-01
  -1.30834013e-01  4.75938544e-02 -2.17004031e-01  2.25336961e-02
   3.76737341e-02  6.38948977e-02 -1.81370288e-01  2.45585412e-01
  -1.25102744e-01 -7.49582332e-03 -7.35026747e-02 -1.38034746e-01
   1.20587610e-01  3.44497025e-01 -9.07964632e-02  5.52004715e-03
  -3.71402

In [91]:
from numpy import dot
from numpy.linalg import norm

def cos_simm(a,b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim
    

In [ ]:
import datetime

start = datetime.datetime.now()

overall_matrix = {}

for item in range(len(numpied)):
    overall_matrix[item] = numpy.zeros(len(numpied))
    for second_item in range(len(numpied)):
        encodings = model.encode([numpied[item], numpied[second_item]])
        overall_matrix[item][second_item] = cos_simm(encodings[0], encodings[1])

end = datetime.datetime.now()

print(end-start)

In [2]:
overall_matrix

NameError: name 'overall_matrix' is not defined

In [ ]:
overall_marix_df = pandas.DataFrame(overall_matrix)
overall_marix_df.to_csv("overall.csv")

0